In [2]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pickle
import pandas as pd
import torch as th
from torch import nn
from torch.autograd import Variable as V

adv_attack_raw_data = pd.read_csv('evaluation/adversarial_attack_data/0323/adv_raw_data_0323_1446.csv')
df = pd.read_csv('evaluation/adversarial_attack_data/0323/adv_data_0323_1446.csv')
df

,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax,IDS_Predicted,True_Label
0,61247.0,2.0,2.0,60.0,2.0,30.0,0.0,132.0,132.0,0.0,...,60.0,0.0,2.0,264.0,-1.0,1.0,0.0,0.0,0.0,1.0
1,113.0,1.0,1.0,6.0,1.0,6.0,0.0,6.0,6.0,0.0,...,6.0,0.0,1.0,6.0,229.0,0.0,0.0,0.0,0.0,1.0
2,4.0,2.0,0.0,12.0,0.0,6.0,0.0,0.0,0.0,0.0,...,12.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,1.0
3,7187773.0,1.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,0.0,...,6.0,0.0,6.0,36.0,0.0,0.0,0.0,45324.0,0.0,1.0
4,98.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,32832.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508795,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2508796,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2508797,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2508798,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [3]:
# collect the data with ids_pred == 1
df_ids_pred_attack = df[df['IDS_Predicted'] == 1]
df_ids_pred_attack

,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax,IDS_Predicted,True_Label
11,65468450.0,2.0,2.0,85.0,2.0,42.0,0.707107,132.0,123.000000,12.727922,...,85.0,0.0,2.0,246.0,-1.0,1.0,0.0,160617.0,1.0,1.0
15,3553499.0,3.0,3.0,0.0,3.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,3.0,0.0,29.0,0.0,0.0,0.0,1.0,1.0
35,1938162.0,6.0,3.0,317.0,3.0,0.0,91.693880,145.0,48.333332,83.715790,...,317.0,0.0,3.0,145.0,110.0,3.0,0.0,0.0,1.0,1.0
49,32032960.0,2.0,0.0,74.0,0.0,37.0,0.000000,0.0,0.000000,0.000000,...,74.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,1.0
52,999757.0,2.0,0.0,12.0,0.0,6.0,0.000000,0.0,0.000000,0.000000,...,12.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508513,67022356.0,2.0,2.0,96.0,2.0,48.0,0.000000,48.0,48.000000,0.000000,...,96.0,0.0,2.0,96.0,-1.0,1.0,0.0,22201.0,1.0,1.0
2508514,2375559.0,10.0,10.0,867.0,10.0,0.0,168.999710,3120.0,1261.500000,1175.215700,...,867.0,0.0,10.0,12615.0,1048.0,3.0,0.0,0.0,1.0,1.0
2508521,20.0,1.0,2.0,6.0,2.0,6.0,0.000000,6.0,6.000000,0.000000,...,6.0,0.0,2.0,12.0,255.0,0.0,0.0,0.0,1.0,1.0
2508528,34.0,1.0,1.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,1.0,0.0,509.0,0.0,0.0,0.0,1.0,1.0


In [4]:
# collect the data with ids_pred == 0 and true_label == 1
df_ids_pred_normal_true_attack = df[df['True_Label'] == 1]
df_ids_pred_normal_true_attack = df_ids_pred_normal_true_attack[df_ids_pred_normal_true_attack['IDS_Predicted'] == 0]
df_ids_pred_normal_true_attack

,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax,IDS_Predicted,True_Label
0,61247.0,2.0,2.0,60.0,2.0,30.0,0.000000,132.0,132.00000,0.00000,...,60.0,0.00,2.0,264.0,-1.0,1.0,0.00,0.0,0.0,1.0
1,113.0,1.0,1.0,6.0,1.0,6.0,0.000000,6.0,6.00000,0.00000,...,6.0,0.00,1.0,6.0,229.0,0.0,0.00,0.0,0.0,1.0
2,4.0,2.0,0.0,12.0,0.0,6.0,0.000000,0.0,0.00000,0.00000,...,12.0,0.00,0.0,0.0,-1.0,1.0,0.00,0.0,0.0,1.0
3,7187773.0,1.0,6.0,6.0,6.0,6.0,0.000000,6.0,6.00000,0.00000,...,6.0,0.00,6.0,36.0,0.0,0.0,0.00,45324.0,0.0,1.0
4,98.0,1.0,1.0,0.0,1.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.0,0.00,1.0,0.0,32832.0,0.0,0.00,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508539,61308412.0,15.0,13.0,1723.0,13.0,0.0,343.348820,1601.0,303.30768,586.86260,...,1723.0,16609.00,13.0,3943.0,120.0,14.0,118862.11,375821.0,0.0,1.0
2508540,118416250.0,28.0,36.0,958.0,36.0,0.0,59.563446,1698.0,970.30554,653.64734,...,958.0,30046.38,36.0,34931.0,360.0,27.0,94071.37,357779.0,0.0,1.0
2508541,305.0,2.0,2.0,66.0,2.0,33.0,0.000000,193.0,193.00000,0.00000,...,66.0,0.00,2.0,386.0,-1.0,1.0,0.00,0.0,0.0,1.0
2508542,24258.0,2.0,2.0,78.0,2.0,39.0,0.000000,148.0,148.00000,0.00000,...,78.0,0.00,2.0,296.0,-1.0,1.0,0.00,0.0,0.0,1.0


In [5]:
adv_attack_raw_data

,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax,IDS_Predicted,True_Label
0,33138628.0,2.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0
1,11571301.0,8.0,6.0,392.0,6.0,0.0,138.59293,4392.0,1938.6666,2190.7188,...,392.0,0.0,6.0,11632.0,235.0,1.0,0.0,933.0,1.0,1.0
2,11809948.0,8.0,6.0,625.0,6.0,0.0,220.97087,5792.0,1938.6666,2544.9190,...,625.0,0.0,6.0,11632.0,237.0,1.0,0.0,571.0,1.0,1.0
3,11500889.0,7.0,5.0,392.0,5.0,0.0,148.16208,10184.0,2326.4000,4437.0560,...,392.0,0.0,5.0,11632.0,235.0,1.0,0.0,914.0,1.0,1.0
4,94057510.0,2.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254395,105080856.0,2.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0
1254396,8450981.0,9.0,4.0,375.0,4.0,0.0,125.00000,7288.0,2908.0000,3566.4840,...,375.0,0.0,4.0,11632.0,235.0,1.0,0.0,845.0,1.0,1.0
1254397,3002821.0,3.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0000,0.0000,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0
1254398,11450899.0,8.0,4.0,300.0,4.0,0.0,106.06602,11632.0,2908.0000,5816.0000,...,300.0,0.0,4.0,11632.0,235.0,1.0,0.0,872.0,1.0,1.0


In [6]:
adv_attack_generate_data = df[df['True_Label'] == 1]
adv_attack_generate_data

,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax,IDS_Predicted,True_Label
0,61247.0,2.0,2.0,60.0,2.0,30.0,0.000000,132.0,132.00000,0.00000,...,60.0,0.00,2.0,264.0,-1.0,1.0,0.00,0.0,0.0,1.0
1,113.0,1.0,1.0,6.0,1.0,6.0,0.000000,6.0,6.00000,0.00000,...,6.0,0.00,1.0,6.0,229.0,0.0,0.00,0.0,0.0,1.0
2,4.0,2.0,0.0,12.0,0.0,6.0,0.000000,0.0,0.00000,0.00000,...,12.0,0.00,0.0,0.0,-1.0,1.0,0.00,0.0,0.0,1.0
3,7187773.0,1.0,6.0,6.0,6.0,6.0,0.000000,6.0,6.00000,0.00000,...,6.0,0.00,6.0,36.0,0.0,0.0,0.00,45324.0,0.0,1.0
4,98.0,1.0,1.0,0.0,1.0,0.0,0.000000,0.0,0.00000,0.00000,...,0.0,0.00,1.0,0.0,32832.0,0.0,0.00,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508539,61308412.0,15.0,13.0,1723.0,13.0,0.0,343.348820,1601.0,303.30768,586.86260,...,1723.0,16609.00,13.0,3943.0,120.0,14.0,118862.11,375821.0,0.0,1.0
2508540,118416250.0,28.0,36.0,958.0,36.0,0.0,59.563446,1698.0,970.30554,653.64734,...,958.0,30046.38,36.0,34931.0,360.0,27.0,94071.37,357779.0,0.0,1.0
2508541,305.0,2.0,2.0,66.0,2.0,33.0,0.000000,193.0,193.00000,0.00000,...,66.0,0.00,2.0,386.0,-1.0,1.0,0.00,0.0,0.0,1.0
2508542,24258.0,2.0,2.0,78.0,2.0,39.0,0.000000,148.0,148.00000,0.00000,...,78.0,0.00,2.0,296.0,-1.0,1.0,0.00,0.0,0.0,1.0


In [9]:
ids_input_df = pd.read_csv("evaluation/adversarial_attack_data/0323/adv_data_0323_1808.csv")
ids_input_df

,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,FwdHeaderLen.1,SubflowFwdPkts,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
124,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
125,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
126,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0


In [1]:
import pandas as pd

df = pd.read_csv('datasets/target_model/CICIDS2017/target_model_CICIDS2017data.csv')
df




,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax,Label,binary_classification
0,119970779,6,8,123,8,6,15.883954,31,13.125000,15.008926,...,123,1.920725e+07,8,105,55480,5,2.133558e+02,32779,BENIGN,benign
1,117591768,99,122,2547,122,0,94.293811,4344,1686.852459,1037.995945,...,2547,1.863627e+03,122,205796,1140,7,7.352316e+05,2453564,BENIGN,benign
2,609363,2,2,37,2,0,26.162951,6,6.000000,0.000000,...,37,0.000000e+00,2,12,0,1,0.000000e+00,0,BENIGN,benign
3,64039358,2,2,96,2,48,0.000000,48,48.000000,0.000000,...,96,0.000000e+00,2,96,-1,1,0.000000e+00,36695,BENIGN,benign
4,57329,1,1,48,1,48,0.000000,48,48.000000,0.000000,...,48,0.000000e+00,1,48,-1,0,0.000000e+00,0,BENIGN,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57567,102645877,14,3,2541,3,0,98.363337,6,2.000000,3.464102,...,2541,1.880000e+07,3,6,0,11,4.529980e+06,6406845,DoS_slowloris,malicious
57568,3003648,3,0,0,0,0,0.000000,0,0.000000,0.000000,...,0,0.000000e+00,0,0,-1,0,0.000000e+00,0,DoS_slowloris,malicious
57569,3003925,3,0,0,0,0,0.000000,0,0.000000,0.000000,...,0,0.000000e+00,0,0,-1,0,0.000000e+00,0,DoS_slowloris,malicious
57570,102690607,14,3,2541,3,0,98.363337,6,2.000000,3.464102,...,2541,1.880000e+07,3,6,0,11,4.532999e+06,6411372,DoS_slowloris,malicious
